In [2]:
import TimeTagger as TT
from TimeTagger import Flim, TimeTagStream, createTimeTaggerVirtual, EventGenerator, Countrate,ConstantFractionDiscriminator, DelayedChannel
from bokeh.models.mappers import LogColorMapper
from bokeh.models import ColorBar, LogTicker

In [3]:
#dump_file = r"Z:\PriyaM\swabian_data\20230712_tissue\test_tissue_2023-07-12_154752.1.ttbin"
time_tagger= TT.createTimeTagger()
#for syncing measurments


In [ ]:
#configs
time_tagger.reset()

#filewriter configs
file_name = pn.widgets.TextInput(name='File Path', placeholder='enter file path')

#flim configs
#check ranges?
laser_num_widg = pn.widgets.IntInput(name='Laser Channel Num', value=1, step=1, start=-10, end=10)
laser = laser_num_widg.value
line_num_widg  = pn.widgets.IntInput(name='Line Channel Num', value=2, step=1, start=-10, end=10)
line = line_num_widg.value
frame_num_widg = pn.widgets.IntInput(name='Frame Channel Num', value=3, step=1, start=-10, end=10)
frame = frame_num_widg.value
photon_num_widg = pn.widgets.IntInput(name='Photon Channel Num', value=-4, step=1, start=-10, end=10)
photon = photon_num_widg.value

n_bins_widg   = pn.widgets.IntInput(name='Number of Bins', value=256, step=1, start=0, end=2048)
#check if bw can be calc?
binwidth_widg = pn.widgets.IntInput(name='Binwidth', value=49, step=1, start=0, end=1000)
#can this just be n_bins?
n_pixel_widg  = pn.widgets.IntInput(name='Number of Pixels', value=256, step=1, start=0, end=2048)
n_pixel = n_pixel_widg.value
#do these need to be set?
pixel_rate_widg = pn.widgets.FloatInput(name='Pixel Rate', value=200e3, step=1e3, start=0, end=400e3)
pixel_time = 1/pixel_rate_widg.value
integ_time_widg = pn.widgets.FloatInput(name='Integration Time', value=3e9, step=1e9, start=0, end=10e9)

duration_widg = pn.widgets.FloatInput(name='Duration (in seconds)', value=1, step=1, start=0, end=100)


#flim dispalay

flim_button = pn.widgets.Button(name='Start Flim Measurement', button_type='primary') #panel widget
flim_widget = pn.panel(figure()) #bokeh wrapper

vmin_widg = pn.widgets.FloatInput(name='vmin', value=1, step=1, start=0, end=100)
vmax_widg = pn.widgets.FloatInput(name='vmax', value=20, step=1, start=0, end=100)
#for vmax and vmin
color_mapper = LogColorMapper(palette="Viridis256", low=vmin_widg.value, high=vmax_widg.value)

#flim class inputs

# EventGenerator(tagger, trigger_channel, pattern, trigger_divider, stop_channel)
#virtual channel
pixel_pattern_start = np.linspace(0, n_pixel, num=n_pixel, endpoint=False, dtype=int) * (pixel_time * 1e12)
pixel_pattern_end = pixel_pattern_start + integ_time_widg.value

pixel_start_chan = TT.EventGenerator(time_tagger, line, pixel_pattern_start)
pixel_end_chan= TT.EventGenerator(time_tagger, line, pixel_pattern_end)

#ConstantFractionDiscriminator(tagger, channels, search_window)
#virtual channel
photon_cfd_chan = TT.ConstantFractionDiscriminator(time_tagger, (photon,), 10*1000)

#DelayedChannel(tagger, input_channel, delay)
#virtual channel
delayed_laser_chan = TT.DelayedChannel(time_tagger, laser, 11.5e3)

#initialize measurement classes
sync = TT.SynchronizedMeasurements(time_tagger)
sync_tagger = sync.getTagger()

flim = TT.Flim(sync_tagger, #register measurement for sync
               start_channel=delayed_laser_chan.getChannel(), 
               click_channel=photon_cfd_chan.getChannels()[0], 
               pixel_begin_channel=pixel_start_chan.getChannel(), 
               n_pixels=n_pixel*n_pixel, 
               n_bins=n_bins_widg.value, 
               binwidth=binwidth_widg.value, 
               pixel_end_channel=pixel_end_chan.getChannel(), 
               frame_begin_channel=frame)

file_writer = TT.FileWriter(sync_tagger, file_name.value, [1,2,3,4])

#on_click callback
def on_flim_button_clicked(b):
    sync.clear()
    
    #start measurement (using sync object)
    sync.startFor(duration_widg.value*1E12)
    sync.waitUntilFinished()

    flim_3d = flim.getSummedFrames().reshape(256,256,256)
    image = flim_3d.sum(2)

    t=figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
    t.image(image=[image],x=0, y=0, dw=image.shape[0], dh=image.shape[1],color_mapper=color_mapper)
    color_bar = ColorBar(color_mapper=color_mapper, 
                         ticker=LogTicker(),
                         label_standoff=12, 
                         border_line_color=None, 
                         location=(0,0))
    t.add_layout(color_bar, 'right')
    flim_widget.object=t #update widget
    
#on_click events
flim_button.on_click(on_flim_button_clicked)

#layout
settings = pn.Column(
    file_name,
    laser_num_widg,
    line_num_widg,
    frame_num_widg,
    photon_num_widg,
    n_bins_widg,  
    binwidth_widg,
    n_pixel_widg,  
    pixel_rate_widg, 
    integ_time_widg
)

flim_app = pn.Row(settings,pn.Column(duration_widg, flim_button, flim_widget),pn.Column(vmin_widg,vmax_widg))